In [1]:
import os
cwd = os.getcwd()
cwd

'/Users/jinlima'

## Step 1: Get Link

In [2]:
import urllib.request
import os,sys,csv,time  #"time" helps to break for the url visiting 
from bs4 import BeautifulSoup   # Need to install this package manually using pip
                                # We only import part of the Beautifulsoup4

os.chdir('/Users/jinlima/Documents/scripts/') # The location of your file "LongCompanyList.csv
companyListFile = "CompanyList.csv" # a csv file with the list of company ticker symbols and names 
                                    # (the file has a line with headers)
IndexLinksFile = "IndexLinks.csv" # a csv file (output of the current script) with the list of index links 
                                  # for each firm (the file has a line with headers)

def getIndexLink(tickerCode,FormType):
    csvOutput = open(IndexLinksFile,"a+b") # "a+b" indicates that we are adding lines rather than replacing lines
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    urlLink = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+tickerCode+"&type="+FormType+"&dateb=&owner=exclude&count=100"
    pageRequest = urllib2.Request(urlLink)
    pageOpen = urllib2.urlopen(pageRequest)
    pageRead = pageOpen.read()
    
    soup = BeautifulSoup(pageRead,"html.parser")
    
    #Check if there is a table to extract / code exists in edgar database
    try:
        table = soup.find("table", { "class" : "tableFile2" })
    except:
        print("No tables found or no matching ticker symbol for ticker symbol for"+tickerCode)
        return -1

    docIndex = 1
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        if len(cells)==5:
            if cells[0].text.strip() == FormType:
                link = cells[1].find("a",{"id": "documentsbutton"})
                docLink = "https://www.sec.gov"+link['href']
                description = cells[2].text.encode('utf8').strip() #strip take care of the space in the beginning and the end
                filingDate = cells[3].text.encode('utf8').strip()
                newfilingDate = filingDate.replace("-","_")  ### <=== Change date format from 2012-1-1 to 2012_1_1 so it can be used as part of 10-K file names
                csvWriter.writerow([tickerCode, docIndex, docLink, description, filingDate,newfilingDate])
                docIndex = docIndex + 1
    csvOutput.close()


def main():  
    FormType = "10-K"   ### <=== Type your document type here
    nbDocPause = 10 ### <=== Type your number of documents to download in one batch
    nbSecPause = 0 ### <=== Type your pausing time in seconds between each batch

    csvFile = open(companyListFile,"r") #<===open and read from a csv file with the list of company 
                                        # ticker symbols (the file has a line with headers)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)
    
    csvOutput = open(IndexLinksFile,"a+b") #<===open and write to a csv file which will include the list of index links. New rows will be appended.
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    csvWriter.writerow(["Ticker", "DocIndex","IndexLink", "Description", "FilingDate","NewFilingDate"])
    csvOutput.close()
    
    i = 1
    for rowData in csvData[1:]:
        ticker = rowData[0]
        getIndexLink(ticker,FormType)
        if i%nbDocPause == 0:
            print(i)
            print("Pause for "+str(nbSecPause)+" second .... ")
            time.sleep(float(nbSecPause))
        i=i+1
       
    csvFile.close()
    print("done!")
    
if __name__ == "__main__":
    main()


TypeError: a bytes-like object is required, not 'str'

## Step 2: 

In [ ]:
import os,sys,csv,urllib2,time
from bs4 import BeautifulSoup   #<---- Need to install this package manually using pip

os.chdir('/Users/alvinzuyinzheng/Dropbox/PythonWorkshop/scripts/')#<===The location of your file "LongCompanyList.csv
IndexLinksFile = "IndexLinks.csv"  #a csv file (output of the 1GetIndexLinks.py script) with the list of index links for each firm (the file has a line with headers)
Form10kListFile = "10kList.csv"    #a csv file (output of the current script) with the list of 10-K links for each firm (the file has a line with headers)

def get10kLink(tickerCode, docIndex, docLink, description, filingDate, newFilingDate, FormType):
    csvOutput = open(Form10kListFile,"a+b")
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    pageRequest = urllib2.Request(docLink)
    pageOpen = urllib2.urlopen(pageRequest)
    pageRead = pageOpen.read()
    
    soup = BeautifulSoup(pageRead,"html.parser")

    #Check if there is a table to extract / code exists in edgar database
    try:
        table = soup.find("table", { "summary" : "Document Format Files" })
    except:
        print "No tables found for link "+docLink
        
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        if len(cells)==5:
            if cells[3].text.strip() == FormType:
                link = cells[2].find("a")
                formLink = "https://www.sec.gov"+link['href']
                formName = link.text.encode('utf8').strip()
                csvWriter.writerow([tickerCode, docIndex, docLink, description, filingDate, newFilingDate, formLink,formName])
    csvOutput.close()


def main():  
    FormType = "10-K"   ### <=== Type your document type here
    nbDocPause = 10 ### <=== Type your number of documents to download in one batch
    nbSecPause = 1 ### <=== Type your pausing time in seconds between each batch

    csvFile = open(IndexLinksFile,"r") #<===Open and read from a csv file with the list of index links for each firm (the file has a line with headers)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)
    
    csvOutput = open(Form10kListFile,"a+b") #<===open and write to a csv file which will include the list of 10-K links. New rows will be appended.
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    csvWriter.writerow(["Ticker", "DocIndex", "IndexLink", "Description", "FilingDate", "NewFilingDate", "Form10KLink","Form10KName"])
    csvOutput.close()
    
    i = 1
    for rowData in csvData[1:]:
        Ticker = rowData[0]
        DocIndex = rowData[1]
        DocLink = rowData[2]
        Description = rowData[3]
        FileDate = rowData[4]
        NewFileDate = rowData[5]
        
        get10kLink(Ticker,DocIndex,DocLink,Description,FileDate,NewFileDate,FormType)
        if i%nbDocPause == 0:
            print i
            print "Pause for "+str(nbSecPause)+" second .... "
            time.sleep(float(nbSecPause))
        i=i+1
       
    csvFile.close()
    print "done!"
    
if __name__ == "__main__":
	main()


In [ ]:
import os,sys,csv,urllib2,time


os.chdir('/Users/alvinzuyinzheng/Dropbox/PythonWorkshop/scripts/')#<===The location of your file "LongCompanyList.csv
htmlSubPath = "./HTML/" #<===The subfolder with the 10-K files in HTML format

Form10kListFile = "10kList.csv" #a csv file (output of the 2Get10kLinks.py script) with the list of 10-K links
logFile = "10kDownloadLog.csv" #a csv file (output of the current script) with the download history of 10-K forms

def dowmload10k(tickerCode, docIndex, docLink, description, filingDate, newFilingDate, formLink,formName):
    csvOutput = open(logFile,"a+b")
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)

    try:
        pageRequest = urllib2.Request(formLink)
        pageOpen = urllib2.urlopen(pageRequest)
        pageRead = pageOpen.read()

        htmlname = tickerCode+"_"+docIndex+"_"+newFilingDate+".htm"
        htmlpath = htmlSubPath+htmlname
        htmlfile = open(htmlpath,'wb')
        htmlfile.write(pageRead)
        htmlfile.close()
        csvWriter.writerow([tickerCode, docIndex, docLink, description, filingDate, newFilingDate, formLink,formName, htmlname, ""])
    except:
        csvWriter.writerow([tickerCode, docIndex, docLink, description, filingDate, newFilingDate, formLink,formName, "","not downloaded"])

    csvOutput.close()

def main():
    if not os.path.isdir(htmlSubPath):  ### <=== keep all HTML files in this subfolder
        os.makedirs(htmlSubPath)
    
    FormType = "10-K"   ### <=== Type your document type here
    nbDocPause = 5 ### <=== Type your number of documents to download in one batch
    nbSecPause = 1 ### <=== Type your pausing time in seconds between each batch

    FormYears = ['2014','2015'] ### <=== Type the years of documents to download here

    csvFile = open(Form10kListFile,"r") #<===A csv file with the list of company ticker symbols (the file has a line with headers)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)

    csvOutput = open(logFile,"a+b")
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)

    csvWriter.writerow(["Ticker", "DocIndex", "IndexLink", "Description", "FilingDate", "NewFilingDate", "Form10KLink","Form10KName", "FileName","Note"])
    csvOutput.close()
    
    i = 1
    for rowData in csvData[1:]:
        Ticker = rowData[0]
        DocIndex = rowData[1]
        IndexLink = rowData[2]
        Description = rowData[3]
        FilingDate = rowData[4]
        NewFilingDate = rowData[5]
        FormLink = rowData[6]
        FormName = rowData[7]
        for year in FormYears:
            if year in FilingDate:
                if ".htm" in FormName:
                    dowmload10k(Ticker, DocIndex, IndexLink, Description, FilingDate, NewFilingDate, FormLink,FormName)
                elif ".txt" in FormName:
                    csvOutput = open(logFile,"a+b")
                    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
                    csvWriter.writerow([Ticker, DocIndex, IndexLink, Description, FilingDate, NewFilingDate, FormLink,FormName, "","Text format"])
                    csvOutput.close()
                else:
                    csvOutput = open(logFile,"a+b")
                    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
                    csvWriter.writerow([Ticker, DocIndex, IndexLink, Description, FilingDate, NewFilingDate, FormLink,FormName,"", "No form"])
                    csvOutput.close()
            
        if i%nbDocPause == 0:
            print i
            print "Pause for "+str(nbSecPause)+" second .... "
            time.sleep(float(nbSecPause))
        i=i+1
       
    csvFile.close()
    print "done!"
    
if __name__ == "__main__":
	main()


In [ ]:
import os, sys, csv, re, urllib2
from bs4 import BeautifulSoup  #<---- Need to install this package manually using pip

os.chdir('/Users/alvinzuyinzheng/Dropbox/PythonWorkshop/scripts/')#<===The location of your csv files
htmlSubPath = "./HTML/" #<===The subfolder with the 10-K files in HTML format
txtSubPath = "./txt/" #<===The subfolder with the extracted text files

DownloadLogFile = "10kDownloadLog.csv" #a csv file (output of the 3DownloadHTML.py script) with the download history of 10-K forms
ReadLogFile = "10kReadlog.csv" #a csv file (output of the current script) showing whether item 1 is successfully extracted from 10-K forms

def readHTML(file_name):
    input_path = htmlSubPath+file_name
    output_path = txtSubPath+file_name.replace(".htm",".txt")
    

    input_file = open(input_path,'rb')
    page = input_file.read()  #<===Read the HTML file into Python


    #Pre-processing the html content by removing extra white space and combining then into one line.
    page = page.strip()  #<=== remove white space at the beginning and end
    page = page.replace('\n', ' ') #<===replace the \n (new line) character with space
    page = page.replace('\r', '') #<===replace the \r (carriage returns -if you're on windows) with space
    page = page.replace('&nbsp;', ' ') #<===replace "&nbsp;" (a special character for space in HTML) with space. 
    page = page.replace('&#160;', ' ') #<===replace "&#160;" (a special character for space in HTML) with space.
    while '  ' in page:
        page = page.replace('  ', ' ') #<===remove extra space

    #Using regular expression to extract texts that match a pattern
        
    #Define pattern for regular expression.
        #The following patterns find ITEM 1 and ITEM 1A as diplayed as subtitles
        #(.+?) represents everything between the two subtitles
    #If you want to extract something else, here is what you should change

    #Define a list of potential patterns to find ITEM 1 and ITEM 1A as subtitles   
    regexs = ('bold;\">\s*Item 1\.(.+?)bold;\">\s*Item 1A\.',   #<===pattern 1: with an attribute bold before the item subtitle
              'b>\s*Item 1\.(.+?)b>\s*Item 1A\.',               #<===pattern 2: with a tag <b> before the item subtitle
              'Item 1\.\s*<\/b>(.+?)Item 1A\.\s*<\/b>',         #<===pattern 3: with a tag <\b> after the item subtitle          
              'Item 1\.\s*Business\.\s*<\/b(.+?)Item 1A\.\s*Risk Factors\.\s*<\/b') #<===pattern 4: with a tag <\b> after the item+description subtitle 

    #Now we try to see if a match can be found...
    for regex in regexs:
        match = re.search (regex, page, flags=re.IGNORECASE)  #<===search for the pattern in HTML using re.search from the re package. Ignore cases.

        #If a match exist....
        if match:
            #Now we have the extracted content still in an HTML format
            #We now turn it into a beautiful soup object
            #so that we can remove the html tags and only keep the texts
            
            soup = BeautifulSoup(match.group(1), "html.parser") #<=== match.group(1) returns the texts inside the parentheses (.*?) 
            

            #soup.text removes the html tags and only keep the texts
            rawText = soup.text.encode('utf8') #<=== you have to change the encoding the unicodes

           
            #remove space at the beginning and end and the subtitle "business" at the beginning
            #^ matches the beginning of the text
            outText = re.sub("^business\s*","",rawText.strip(),flags=re.IGNORECASE)
            
            output_file = open(output_path, "w")
            output_file.write(outText)  
            output_file.close()
            
            break  #<=== if a match is found, we break the for loop. Otherwise the for loop continues

    input_file.close()    

    return match

def main():

    if not os.path.isdir(txtSubPath):  ### <=== keep all texts files in this subfolder
        os.makedirs(txtSubPath)
        
    csvFile = open(DownloadLogFile, "rb") #<===A csv file with the list of 10k file names (the file should have no header)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)
    
    logFile = open(ReadLogFile, "a+b") #<===A log file to track which file is successfully extracted
    logWriter = csv.writer(logFile, quoting = csv.QUOTE_NONNUMERIC)
    logWriter.writerow(["filename","extracted"])

    i=1
    for rowData in csvData[1:]:
##        Ticker = rowData[0]
##        DocIndex = rowData[1]
##        IndexLink = rowData[2]
##        Description = rowData[3]
##        FilingDate = rowData[4]
##        NewFilingDate = rowData[5]
##        FormLink = rowData[6]
##        FormName = rowData[7]
        FileName = rowData[8]
        
        if ".htm" in FileName:        
            match=readHTML(FileName)
            if match:
                logWriter.writerow([FileName,"yes"])
            else:
                logWriter.writerow([FileName,"no"])
        i=i+1
        
    csvFile.close()

    logFile.close()
    print "done!"
            

if __name__ == "__main__":
    main()


## Web data extract

In [ ]:
import os,sys,csv,urllib2,time #"time" helps to break for the url visiting 
from bs4 import BeautifulSoup   # Need to install this package manually using pip
                                # We only import part of the Beautifulsoup4

os.chdir('/Users/alvinzuyinzheng/Dropbox/PythonWorkshop/scripts/')# The location of your file "LongCompanyList.csv
companyListFile = "CompanyList.csv" # a csv file with the list of company ticker symbols and names (the file has a line with headers)
IndexLinksFile = "IndexLinks.csv" # a csv file (output of the current script) with the list of index links for each firm (the file has a line with headers)

def getIndexLink(tickerCode,FormType):
    csvOutput = open(IndexLinksFile,"a+b") # "a+b" indicates that we are adding lines rather than replacing lines
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    urlLink = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK="+tickerCode+"&type="+FormType+"&dateb=&owner=exclude&count=100"
    pageRequest = urllib2.Request(urlLink)
    pageOpen = urllib2.urlopen(pageRequest)
    pageRead = pageOpen.read()
    
    soup = BeautifulSoup(pageRead,"html.parser")
    
    #Check if there is a table to extract / code exists in edgar database
    try:
        table = soup.find("table", { "class" : "tableFile2" })
    except:
        print "No tables found or no matching ticker symbol for ticker symbol for"+tickerCode
        return -1

    docIndex = 1
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        if len(cells)==5:
            if cells[0].text.strip() == FormType:
                link = cells[1].find("a",{"id": "documentsbutton"})
                docLink = "https://www.sec.gov"+link['href']
                description = cells[2].text.encode('utf8').strip() #strip take care of the space in the beginning and the end
                filingDate = cells[3].text.encode('utf8').strip()
                newfilingDate = filingDate.replace("-","_")  ### <=== Change date format from 2012-1-1 to 2012_1_1 so it can be used as part of 10-K file names
                csvWriter.writerow([tickerCode, docIndex, docLink, description, filingDate,newfilingDate])
                docIndex = docIndex + 1
    csvOutput.close()


def main():  
    FormType = "10-K"   ### <=== Type your document type here
    nbDocPause = 10 ### <=== Type your number of documents to download in one batch
    nbSecPause = 0 ### <=== Type your pausing time in seconds between each batch

    csvFile = open(companyListFile,"r") #<===open and read from a csv file with the list of company ticker symbols (the file has a line with headers)
    csvReader = csv.reader(csvFile,delimiter=",")
    csvData = list(csvReader)
    
    csvOutput = open(IndexLinksFile,"a+b") #<===open and write to a csv file which will include the list of index links. New rows will be appended.
    csvWriter = csv.writer(csvOutput, quoting = csv.QUOTE_NONNUMERIC)
    
    csvWriter.writerow(["Ticker", "DocIndex","IndexLink", "Description", "FilingDate","NewFilingDate"])
    csvOutput.close()
    
    i = 1
    for rowData in csvData[1:]:
        ticker = rowData[0]
        getIndexLink(ticker,FormType)
        if i%nbDocPause == 0:
            print i
            print "Pause for "+str(nbSecPause)+" second .... "
            time.sleep(float(nbSecPause))
        i=i+1
       
    csvFile.close()
    print "done!"
    
if __name__ == "__main__":
	main()
